In [1]:
conda install geopandas


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None

ModuleNotFoundError: No module named 'geopandas'

In [ ]:
from map import extract_html_table, get_coords_by_code

In [ ]:
geojson_raw = './data/national-map-line-geojson.geojson'
map_gpd = gpd.read_file(geojson_raw)

map_gpd = map_gpd.apply(extract_html_table, axis=1)
map_gpd = map_gpd.drop(columns=["Description"])

c1 = map_gpd["Folderpath"] == "Layers/Expressway"
c2 = map_gpd["Folderpath"] == "Layers/Major_Road"

map_clean = map_gpd[c1 | c2]

In [ ]:
coordinates = pd.read_csv('out/coordinates.csv', low_memory=False, lineterminator='\n')
mrt_coordinates = coordinates[coordinates["type"] == "mrt"]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
map_clean["geometry"].plot(ax=ax, alpha=0.25)
plt.title("Singapore Map")

for i, row in mrt_coordinates.iterrows():
    latitude = row["latitude"]
    longitude = row["longitude"]

    code = row["station_code"]
    color = "black"

    if ("NS" in code): color = "red"
    elif ("CC" in code): color = "orange"
    elif ("DT" in code): color = "blue"
    elif ("EW" in code): color = "green"
    elif ("NE" in code): color = "purple"

    plt.plot(
      longitude,
      latitude,
      markersize=5,
      color=color,
      marker='s',
      alpha=0.75
    )

plt.show()

In [3]:
import numpy as np
from sklearn.metrics import DistanceMetric

df = mrt_coordinates[["station_code", "latitude", "longitude"]].iloc[20:35]

df['latitude'] = np.radians(df['latitude'])
df['longitude'] = np.radians(df['longitude'])

dist = DistanceMetric.get_metric('haversine')

matrix = dist.pairwise(df[["latitude", "longitude"]].to_numpy()) * 6373

matrix_df = pd.DataFrame(matrix, columns=df["station_code"].unique(), index=df["station_code"].unique())

ImportError: cannot import name 'DistanceMetric' from 'sklearn.metrics' (C:\Users\pc\anaconda3\lib\site-packages\sklearn\metrics\__init__.py)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
map_clean["geometry"].plot(ax=ax, alpha=0.10)
plt.title("Singapore Map")

for i, row in mrt_coordinates.iterrows():
    latitude = row["latitude"]
    longitude = row["longitude"]

    code = row["station_code"]
    color = "black"

    if ("NS" in code): color = "red"
    elif ("CC" in code): color = "orange"
    elif ("DT" in code): color = "blue"
    elif ("EW" in code): color = "green"
    elif ("NE" in code): color = "purple"

    plt.plot(longitude, latitude, markersize=5, color=color, marker='s', alpha=0.50)

for destination, row in matrix_df.iterrows():
  source = "NS23"

  distance = row[source]

  [lat1, lon1] = get_coords_by_code(mrt_coordinates, source)
  [lat2, lon2] = get_coords_by_code(mrt_coordinates, destination)

  x_values = [lon1, lon2]
  y_values = [lat1, lat2]

  if (distance > 10):
    plt.text(lon2, lat2, f"{round(distance)}km", size=11, weight='bold', color="purple")

  plt.plot(x_values, y_values, linestyle="-", alpha=0.5, color="black")

plt.show()

In [ ]:
from python_tsp.exact import solve_tsp_dynamic_programming

permutation, distance = solve_tsp_dynamic_programming(matrix_df.to_numpy())

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
map_clean["geometry"].plot(ax=ax, alpha=0.10)
plt.title("Singapore Map")

for i, row in mrt_coordinates.iterrows():
    latitude = row["latitude"]
    longitude = row["longitude"]

    code = row["station_code"]
    color = "black"

    if ("NS" in code): color = "red"
    elif ("CC" in code): color = "orange"
    elif ("DT" in code): color = "blue"
    elif ("EW" in code): color = "green"
    elif ("NE" in code): color = "purple"

    plt.plot(longitude, latitude, markersize=5, color=color, marker='s', alpha=0.50)

prev = None
for index in permutation:
  if (prev == None):
    prev = index
  else:
    prev_node = matrix_df.index[prev]
    node = matrix_df.index[index]

    [lat1, lon1] = get_coords_by_code(mrt_coordinates, prev_node)
    [lat2, lon2] = get_coords_by_code(mrt_coordinates, node)

    x_values = [lon1, lon2]
    y_values = [lat1, lat2]
    plt.plot(x_values, y_values, linestyle="-", alpha=1, linewidth=3, color="black")

    prev = index
  
plt.show()

print(f"{distance} km")

In [5]:
pip install py2opt

  Created wheel for random2: filename=random2-1.0.1-py3-none-any.whl size=12075 sha256=dcab96d18fd145719fcb00994cf0dea09877d8ff70561e9d9e95f37cac3d9956
  Stored in directory: c:\users\pc\appdata\local\pip\cache\wheels\6e\ca\f2\082dec051ffcaec249ae491b8c90e305726a7390274682b4a4
Successfully built random2



In [6]:
from py2opt.routefinder import RouteFinder

route_finder = RouteFinder(matrix_df.values.tolist(), matrix_df.index.tolist(), iterations=10)
best_distance, best_route = route_finder.solve()

print(best_distance)
print(best_route)

NameError: name 'matrix_df' is not defined

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
map_clean["geometry"].plot(ax=ax, alpha=0.10)
plt.title("Singapore Map")

for i, row in mrt_coordinates.iterrows():
    latitude = row["latitude"]
    longitude = row["longitude"]

    code = row["station_code"]
    color = "black"

    if ("NS" in code): color = "red"
    elif ("CC" in code): color = "orange"
    elif ("DT" in code): color = "blue"
    elif ("EW" in code): color = "green"
    elif ("NE" in code): color = "purple"

    plt.plot(longitude, latitude, markersize=5, color=color, marker='s', alpha=0.50)

prev_station = None
for station in best_route:
  if (prev_station == None):
    prev_station = station
  else:
    [lat1, lon1] = get_coords_by_code(mrt_coordinates, prev_station)
    [lat2, lon2] = get_coords_by_code(mrt_coordinates, station)

    x_values = [lon1, lon2]
    y_values = [lat1, lat2]
    plt.plot(x_values, y_values, linestyle="-", alpha=1, linewidth=3, color="black")

    prev_station = station
  
plt.show()

print(f"{best_distance} km")